In [ ]:
from simulation.model import PathFindingModel
import mesa
import analytics.ranking_measures as measures
import pandas as pd
from analytics.prob_measures import *

In [ ]:
def group_summary(df, measure_func):
    group_statistics = {
        'zone':df['goal_zones'].iloc[0],
        'alg':df['path_planning_alg'].iloc[0],
        'goal_pop':df['intention_num'].iloc[0],
        'seed':df['seed'].iloc[0],
        'obstacle_used':df['obstacle_used'].iloc[0],
        'segments':df['segment_num'].max(),
        'segmentation':measure_func(df['Segment_ranking'],ground_truth_position=df['true_intention'].iloc[0]),
        'masters':measure_func(df['Masters_ranking'],ground_truth_position=df['true_intention'].iloc[0]),
        'mirroring':measure_func(df['Mirroring_ranking'],ground_truth_position=df['true_intention'].iloc[0]),
        'segmentation_probs':df['Segment_probs'].iloc[-1],
        'masters_probs':df['Masters_probs'].iloc[-1],
        'mirroring_probs':df['Mirroring_probs'].iloc[-1],
        'segmentation_time':df['Segment_step_time'].mean(),
        'fast_segmentation_time':df['FastSegment_step_time'].mean(),
        'masters_time':df['Masters_step_time'].mean(),
        'mirroring_time':df['Mirroring_step_time'].mean()}
    return pd.Series(group_statistics)

def df_transform(raw_df, measure_func):
    run_statistics = raw_df.groupby('RunId').apply(lambda x: group_summary(x, measure_func))
    # run_statistics.set_index(['zone','goal_pop','seed','alg','obstacle_used'], inplace=True)
    return run_statistics


## Experiment 1: the impact of number of intentions

### Single goal, two intentions: to the true destination with or without obstacles


In [ ]:
params = {"width":20,"height":20,"obs_num":40,"goal_zones":'0',"seed":range(0,1000),"path_planning_alg":0,"intention_profile":1}
results = mesa.batch_run(
    PathFindingModel,
    parameters = params,
    iterations = 1,
    max_steps = 500,
    number_processes = None,
    data_collection_period = 1,
    display_progress=True,
)
results_df_e1_1 = pd.DataFrame(results)

In [ ]:
# results_df_e1_1.to_csv('results/e1_1_raw.csv')

In [ ]:
# results_df_e1_1 = pd.read_csv('results/e1_1_raw.csv')

In [ ]:
# results_df_e1_1

In [ ]:
e1_1_final_correctness = df_transform(results_df_e1_1, measures.final_correctness)
e1_1_fc_long = pd.melt(e1_1_final_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Final Correctness')
e1_1_fc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e1_1_gc = df_transform(results_df_e1_1, measures.correctness)
e1_1_gc_long = pd.melt(e1_1_gc.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Overall Correctness')
e1_1_gc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e1_1_cc = df_transform(results_df_e1_1, measures.last_change)
e1_1_cc_long = pd.melt(e1_1_cc.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Convergence Speed')
e1_1_cc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e1_1_stability = df_transform(results_df_e1_1, measures.stability)
e1_1_stability_long = pd.melt(e1_1_stability.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Stability')
e1_1_stability_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e1_1 = pd.concat([e1_1_fc_long, e1_1_gc_long, e1_1_cc_long, e1_1_stability_long], axis=1)
e1_1.reset_index(names=['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e1_1_long = pd.melt(e1_1, id_vars=['zone','goal_pop','seed','alg','obstacle_used','measure'], value_vars=['Final Correctness','Overall Correctness','Convergence Speed','Stability'], var_name='metric', value_name='value')

In [ ]:
# e1_1_long.to_csv('results/e1_1_cost.csv')

In [ ]:
# e1_1_long = pd.read_csv('results/e1_1.csv')

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(font_scale=1.2)
with sns.axes_style("whitegrid"):
    g = sns.FacetGrid(e1_1_long, col="metric", height=3, aspect = 0.9, col_order=['General Correctness','Final Correctness','Convergence Correctness','Stability'])
g.map(sns.violinplot,"measure", "value", "obstacle_used",split=True, cut=0, order=['segmentation','masters','mirroring'],hue_order=[False,True],palette='Set2',linewidth=1,inner='quartile',scale='width',x_label_rotation=20)
g.set_titles("{col_name}")
g.add_legend(title='Obstacle Used')


In [ ]:
# g.savefig('results/e1_1.pdf', format='pdf')

In [ ]:
# e1_1_long

### separate visualization

In [ ]:
tmp = e1_1_long.replace({'obstacle_used':{False:'No Obstacle',True:'Obstacle Used'}})

#### obstacle not used

In [ ]:
import seaborn as sns
sns.set(font='Arial',font_scale=1.4)
g = sns.FacetGrid(tmp, row='obstacle_used',col="metric", height=3, col_order=['Overall Correctness','Final Correctness','Convergence Speed','Stability'], row_order=['No Obstacle','Obstacle Used'], 
                  margin_titles=True, 
                  palette='Set2',
                  hue='measure',
                  hue_order = ['segmentation','masters','mirroring']
                  )
g.set_titles(row_template="{row_name}",col_template="{col_name}")
g.map(sns.violinplot,"measure", "value", 
      cut=0, 
      order=['segmentation','masters','mirroring'], 
      dodge=False,
      inner=None,
      bw_adjust=0.2,
      density_norm = "area",
      )
g.map(sns.pointplot,"measure", "value",color='black',errorbar=None,estimator='mean',order=['segmentation','masters','mirroring'])
g.set_xlabels('')
g.set_xticklabels(['','',''])



In [ ]:
# g.savefig('results/e1_1.pdf', format='pdf')

#### computation time

In [ ]:
import seaborn as sns 
sns.set_theme(font_scale=1.5)
e1_general_correctness = df_transform(results_df_e1_1, measures.correctness)
e1_time_long = pd.melt(e1_general_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation_time','fast_segmentation_time','masters_time','mirroring_time'], var_name='measure', value_name='Time Cost')
e1_time_long['Time Cost'] = e1_time_long['Time Cost'] * 1000

e1_time_long = e1_time_long[e1_time_long['obstacle_used']==True]

with sns.axes_style("whitegrid"):
    g= sns.catplot(data=e1_time_long, x='measure', y='Time Cost', kind='box', color='grey')
g.set(ylim=(0, 30))
g.set_xticklabels(['Segmentation','Fast Segmentation','Masters','Mirroring'], rotation=20, ha='right')
g.set(ylabel='Time Cost (ms)', xlabel=None)

In [ ]:
# e1_time_long

In [ ]:
# g.savefig('results/tkde/e1_time2.pdf', format='pdf')


### Multiple goals, variable number of intentions: adding potential destinations

In [ ]:
params = {"width":20,"height":20,"obs_num":40,"goal_zones":['0,0','0,0,0,0,0,0','0,0,0,0,0,0,0,0,0,0'],"seed":range(1500,2500),"path_planning_alg":2,"intention_profile":1}
results = mesa.batch_run(
    PathFindingModel,
    parameters = params,
    iterations = 1,
    max_steps = 500,
    number_processes = None,
    data_collection_period = 1,
    display_progress=True,
)
results_df_e1_2 = pd.DataFrame(results)
# results_df_e1_2.to_csv('results/e1_2_raw.csv')

In [ ]:
# results_df_e1_2 = pd.read_csv('results/e1_2_raw.csv')

In [ ]:
e1_2_final_correctness = df_transform(results_df_e1_2, measures.final_correctness)
e1_2_final_correctness['destinition_pop'] = e1_2_final_correctness['zone'].apply(lambda x: len(x.split(',')))
e1_2_fc_long = pd.melt(e1_2_final_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','destinition_pop','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Final Correctness')
e1_2_fc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','destinition_pop','segments','measure'], inplace=True)
e1_2_general_correctness = df_transform(results_df_e1_2, measures.correctness)
e1_2_general_correctness['destinition_pop'] = e1_2_general_correctness['zone'].apply(lambda x: len(x.split(',')))
e1_2_gc_long = pd.melt(e1_2_general_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','destinition_pop','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Overall Correctness')
e1_2_gc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','destinition_pop','segments','measure'], inplace=True)
e1_2_convergence_correctness = df_transform(results_df_e1_2, measures.last_change)
e1_2_convergence_correctness['destinition_pop'] = e1_2_convergence_correctness['zone'].apply(lambda x: len(x.split(',')))
e1_2_cc_long = pd.melt(e1_2_convergence_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','destinition_pop','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Convergence Speed')
e1_2_cc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','destinition_pop','segments','measure'], inplace=True)
e1_2_stability = df_transform(results_df_e1_2, measures.stability)
e1_2_stability['destinition_pop'] = e1_2_stability['zone'].apply(lambda x: len(x.split(',')))
e1_2_stability_long = pd.melt(e1_2_stability.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','destinition_pop','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Stability')
e1_2_stability_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','destinition_pop','segments','measure'], inplace=True)
e1_2 = pd.concat([e1_2_fc_long, e1_2_gc_long, e1_2_cc_long, e1_2_stability_long], axis=1)
e1_2.reset_index(names=['zone','goal_pop','seed','alg','obstacle_used','destinition_pop','segments','measure'], inplace=True)
e1_2_long = pd.melt(e1_2, id_vars=['zone','goal_pop','seed','alg','obstacle_used','destinition_pop','segments','measure'], value_vars=['Final Correctness','Overall Correctness','Convergence Speed','Stability'], var_name='metric', value_name='value')


In [ ]:
# e1_2_long.to_csv('results/e1_2.csv')

In [ ]:
# e1_2_long = pd.read_csv('results/e1_2.csv')

In [ ]:
tmp = e1_2_long[e1_2_long['obstacle_used']==True]

In [ ]:
import seaborn as sns 
sns.set_theme()
g = sns.FacetGrid(tmp,row="destinition_pop", col="metric", height=3, col_order=['Overall Correctness','Final Correctness','Convergence Speed','Stability'], margin_titles=True,
                  palette='Set2',
                  hue='measure',
                  hue_order = ['segmentation','masters','mirroring'])
# g.set_titles(row_template="{row_name} destinations",col_template="{col_name}")
g.set_titles(row_template="",col_template="")
g.map(sns.violinplot,"measure", "value", 
      cut=0, 
      order=['segmentation','masters','mirroring'], 
      dodge=False,
      inner=None,
      bw_adjust=0.2,
      density_norm = "area",
      )
g.map(sns.pointplot,"measure", "value",color='black',errorbar=None,estimator='mean',order=['segmentation','masters','mirroring'])
g.set_xlabels('')
g.set_xticklabels(['','',''])
g.set_ylabels('')
g.tick_params(axis='both', which='major', labelsize=25)

In [ ]:
# g.savefig('results/tkde/e1_2.pdf', format='pdf')

T-test to confirm the significance of the difference in final correctness

In [ ]:
tmp['destinition_pop'].unique()

In [ ]:
from scipy.stats import ttest_ind

In [ ]:
destination6_fc_segmentation = e1_2_long[(e1_2_long['obstacle_used']==True) & (e1_2_long['measure']=='segmentation') & (e1_2_long['metric']=='Final Correctness') & (e1_2_long['destinition_pop']==6)]
destination6_fc_masters = e1_2_long[(e1_2_long['obstacle_used']==True) & (e1_2_long['measure']=='masters') & (e1_2_long['metric']=='Final Correctness') & (e1_2_long['destinition_pop']==6)]
destination6_fc_mirroring = e1_2_long[(e1_2_long['obstacle_used']==True) & (e1_2_long['measure']=='mirroring') & (e1_2_long['metric']=='Final Correctness') & (e1_2_long['destinition_pop']==6)]
t_stat1, p_value1 = ttest_ind(destination6_fc_segmentation['value'], destination6_fc_masters['value'])
print(t_stat1, p_value1)
t_stat2, p_value2 = ttest_ind(destination6_fc_segmentation['value'], destination6_fc_mirroring['value'])
print(t_stat2, p_value2)

In [ ]:
destination9_fc_segmentation = e1_2_long[(e1_2_long['obstacle_used']==True) & (e1_2_long['measure']=='segmentation') & (e1_2_long['metric']=='Final Correctness') & (e1_2_long['destinition_pop']==10)]
destination9_fc_masters = e1_2_long[(e1_2_long['obstacle_used']==True) & (e1_2_long['measure']=='masters') & (e1_2_long['metric']=='Final Correctness') & (e1_2_long['destinition_pop']==10)]
destination9_fc_mirroring = e1_2_long[(e1_2_long['obstacle_used']==True) & (e1_2_long['measure']=='mirroring') & (e1_2_long['metric']=='Final Correctness') & (e1_2_long['destinition_pop']==10)]
t_stat3, p_value3 = ttest_ind(destination9_fc_segmentation['value'], destination9_fc_masters['value'])
print(t_stat3, p_value3)
t_stat4, p_value4 = ttest_ind(destination9_fc_segmentation['value'], destination9_fc_mirroring['value'])
print(t_stat4, p_value4)

In [ ]:
import seaborn as sns
sns.set(font='Arial',font_scale=1.4)
g = sns.FacetGrid(tmp, col="metric", col_order=['Overall Correctness','Final Correctness','Convergence Speed','Stability'], 
                  margin_titles=True, 
                  palette='Set2',
                  hue='measure',
                  hue_order = ['segmentation','masters','mirroring']
                  )
g.set_titles(row_template="{row_name}",col_template="{col_name}")
g.map(sns.lineplot, 'segments', 'value')
g.set(ylim=(0,1),xlim=(0,20))

## Experiment 2: the impact of destination destribution
comparing three scenarios:
1. Uniform distribution in zone 2 only
2. Uniform distribution in zone 1,2,3
3. Uniform distribution in zone 0

In [ ]:
params = {"width":20,"height":20,"obs_num":40,"goal_zones":['2,0,0,0','2,2,2,2','2,1,2,3'],"seed":range(1,1000),"path_planning_alg":0,"intention_profile":1}
results = mesa.batch_run(
    PathFindingModel,
    parameters = params,
    iterations = 1,
    max_steps = 1000,
    number_processes = None,
    data_collection_period = 1,
    display_progress=True,
)
results_df_e2 = pd.DataFrame(results)
# results_df_e2.to_csv('results/e2_raw.csv')

In [ ]:
# results_df_e2 = pd.read_csv('results/e2_raw.csv')

In [ ]:
e2_final_correctness = df_transform(results_df_e2, measures.final_correctness)
e2_fc_long  = pd.melt(e2_final_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Final Correctness')
e2_fc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','segments','measure'], inplace=True)
e2_general_correctness = df_transform(results_df_e2, measures.correctness)
e2_gc_long  = pd.melt(e2_general_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Overall Correctness')
e2_gc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','segments','measure'], inplace=True)
e2_convergence_correctness = df_transform(results_df_e2, measures.last_change)
e2_cc_long  = pd.melt(e2_convergence_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Convergence Speed')
e2_cc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','segments','measure'], inplace=True)
e2_stability = df_transform(results_df_e2, measures.stability)
e2_stability_long  = pd.melt(e2_stability.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used','segments'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Stability')
e2_stability_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','segments','measure'], inplace=True)
e2 = pd.concat([e2_fc_long, e2_gc_long, e2_cc_long, e2_stability_long], axis=1)
e2.reset_index(names=['zone','goal_pop','seed','alg','obstacle_used','segments','measure'], inplace=True)
e2_long = pd.melt(e2, id_vars=['zone','goal_pop','seed','alg','obstacle_used','segments','measure'], value_vars=['Final Correctness','Overall Correctness','Convergence Speed','Stability'], var_name='metric', value_name='value')

In [ ]:
# e2_long.to_csv('results/e2.csv')

In [ ]:
# e2_long = pd.read_csv('results/e2.csv')

In [ ]:
e2_long['zone'] = e2_long['zone'].replace({'0,0,0,0':'sparse','2,2,2,2':'dense','2,1,2,3':'medium'})
e2_long.replace(['General Correctness','Convergence Correctness'], ['Overall Correctness','Convergence Speed'], inplace=True)
e2_long.loc[e2_long['metric']=='Convergence Speed','value'] = 1 - e2_long.loc[e2_long['metric']=='Convergence Speed','value']
tmp = e2_long[e2_long['obstacle_used']==True]

In [ ]:
import seaborn as sns
sns.set_theme()
g = sns.FacetGrid(tmp,row="zone", row_order=['sparse','medium','dense'], col="metric", height=3, col_order=['Overall Correctness','Final Correctness','Convergence Speed','Stability'], margin_titles=True,
                  palette='Set2',
                  hue='measure',
                  hue_order = ['segmentation','masters','mirroring'])
g.set_titles(row_template="",col_template="")
g.map(sns.violinplot,"measure", "value", 
      cut=0, 
      order=['segmentation','masters','mirroring'], 
      dodge=False,
      inner=None,
      bw_adjust=0.2,
      density_norm = "area",
      )
g.map(sns.pointplot,"measure", "value",color='black',errorbar=None,estimator='mean',order=['segmentation','masters','mirroring'])
g.set_xlabels('')
g.set_xticklabels(['','',''])
g.set_ylabels('')
g.tick_params(axis='both', which='major', labelsize=25)

In [ ]:
# g.savefig('results/tkde/e2.pdf', format='pdf')

T-test to confirm the significance of the difference in final correctness

In [ ]:
# tmp

In [ ]:
sparse_fc_segmentation = e2_long[(e2_long['measure']=='segmentation') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='sparse')]
sparse_fc_masters = e2_long[(e2_long['measure']=='masters') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='sparse')]
sparse_fc_mirroring = e2_long[(e2_long['measure']=='mirroring') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='sparse')]
t_stat1, p_value1 = ttest_ind(sparse_fc_segmentation['value'], sparse_fc_masters['value'])
print(t_stat1, p_value1)
t_stat2, p_value2 = ttest_ind(sparse_fc_segmentation['value'], sparse_fc_mirroring['value'])
print(t_stat2, p_value2)

In [ ]:
medium_fc_segmentation = e2_long[(e2_long['measure']=='segmentation') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='medium')]
medium_fc_masters = e2_long[(e2_long['measure']=='masters') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='medium')]
medium_fc_mirroring = e2_long[(e2_long['measure']=='mirroring') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='medium')]
t_stat1, p_value1 = ttest_ind(medium_fc_segmentation['value'], medium_fc_masters['value'])
print(t_stat1, p_value1)
t_stat2, p_value2 = ttest_ind(medium_fc_segmentation['value'], medium_fc_mirroring['value'])
print(t_stat2, p_value2)

In [ ]:
dense_fc_segmentation = e2_long[(e2_long['measure']=='segmentation') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='dense')]
dense_fc_masters = e2_long[(e2_long['measure']=='masters') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='dense')]
dense_fc_mirroring = e2_long[(e2_long['measure']=='mirroring') & (e2_long['metric']=='Final Correctness') & (e2_long['zone']=='dense')]
t_stat3, p_value3 = ttest_ind(dense_fc_segmentation['value'], dense_fc_masters['value'])
print(t_stat3, p_value3)
t_stat4, p_value4 = ttest_ind(dense_fc_segmentation['value'], dense_fc_mirroring['value'])
print(t_stat4, p_value4)

In [ ]:
import seaborn as sns
sns.set(font='Arial',font_scale=1.4)
g = sns.FacetGrid(e2_long, col="metric", col_order=['Overall Correctness','Final Correctness','Convergence Speed','Stability'], 
                  margin_titles=True, 
                  palette='Set2',
                  hue='measure',
                  hue_order = ['segmentation','masters','mirroring']
                  )
g.set_titles(row_template="{row_name}",col_template="{col_name}")
g.map(sns.lineplot, 'segments', 'value')
g.set(ylim=(0,1),xlim=(0,20))

## Experiment 3: the impact of movement patterns

In [ ]:
params = {"width":20,"height":20,"obs_num":40,"goal_zones":'2,2,2,2',"seed":range(0,1000),"path_planning_alg":[0,1,2],"intention_profile":1}
results = mesa.batch_run(
    PathFindingModel,
    parameters = params,
    iterations = 1,
    max_steps = 1000,
    number_processes = None,
    data_collection_period = 1,
    display_progress=True,
)
results_df_e3 = pd.DataFrame(results)

In [ ]:
e3_final_correctness = df_transform(results_df_e3, measures.final_correctness)
e3_fc_long  = pd.melt(e3_final_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Final Correctness')
e3_fc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e3_general_correctness = df_transform(results_df_e3, measures.correctness)
e3_gc_long  = pd.melt(e3_general_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='General Correctness')
e3_gc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e3_convergence_correctness = df_transform(results_df_e3, measures.last_change)
e3_cc_long  = pd.melt(e3_convergence_correctness.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Convergence Correctness')
e3_cc_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e3_stability = df_transform(results_df_e3, measures.stability)
e3_stability_long  = pd.melt(e3_stability.reset_index(), id_vars=['zone','alg','goal_pop','seed','obstacle_used'], value_vars=['segmentation','masters','mirroring'], var_name='measure', value_name='Stability')
e3_stability_long.set_index(['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e3 = pd.concat([e3_fc_long, e3_gc_long, e3_cc_long, e3_stability_long], axis=1)
e3.reset_index(names=['zone','goal_pop','seed','alg','obstacle_used','measure'], inplace=True)
e3_long = pd.melt(e3, id_vars=['zone','goal_pop','seed','alg','obstacle_used','measure'], value_vars=['Final Correctness','General Correctness','Convergence Correctness','Stability'], var_name='metric', value_name='value')

In [ ]:
# e3_long.to_csv('results/e3.csv')

In [ ]:
# e3_long = pd.read_csv('results/e3.csv')

In [ ]:
e3_long['alg'] = e3_long['alg'].replace({0:'DWA',1:'ASTAR',2:'BUG0'})
tmp = e3_long.replace(['General Correctness','Convergence Correctness'], ['Overall Correctness','Convergence Speed'])
tmp.loc[tmp['metric']=='Convergence Speed','value'] = 1 - tmp.loc[tmp['metric']=='Convergence Speed','value']
tmp = tmp[tmp['obstacle_used']==True]

In [ ]:
import seaborn as sns
sns.set_theme()
g = sns.FacetGrid(tmp,row="alg", row_order=['ASTAR','DWA','BUG0'], col="metric", height=3, col_order=['Overall Correctness','Final Correctness','Convergence Speed','Stability'], margin_titles=True,
                  palette='Set2',
                  hue='measure',
                  hue_order = ['segmentation','masters','mirroring'])
g.set_titles(row_template="",col_template="")
g.map(sns.violinplot,"measure", "value", 
      cut=0, 
      order=['segmentation','masters','mirroring'], 
      dodge=False,
      inner=None,
      bw_adjust=0.2,
      density_norm = "area",
      )
g.map(sns.pointplot,"measure", "value",color='black',errorbar=None,estimator='mean',order=['segmentation','masters','mirroring'])
g.set_xlabels('')
g.set_xticklabels(['','',''])
g.set_ylabels('')
g.tick_params(axis='both', which='major', labelsize=25)

In [ ]:
# g.savefig('results/tkde/e3.pdf', format='pdf')

T-test to confirm the significance of the difference in final correctness

In [ ]:
astar_fc_segmentation = e3_long[(e3_long['measure']=='segmentation') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='ASTAR')]
astar_fc_masters = e3_long[(e3_long['measure']=='masters') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='ASTAR')]
astar_fc_mirroring = e3_long[(e3_long['measure']=='mirroring') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='ASTAR')]
t_stat1, p_value1 = ttest_ind(astar_fc_segmentation['value'], astar_fc_masters['value'])
print(t_stat1, p_value1)
t_stat2, p_value2 = ttest_ind(astar_fc_segmentation['value'], astar_fc_mirroring['value'])
print(t_stat2, p_value2)

In [ ]:
dwa_fc_segmentation = e3_long[(e3_long['measure']=='segmentation') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='DWA')]
dwa_fc_masters = e3_long[(e3_long['measure']=='masters') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='DWA')]
dwa_fc_mirroring = e3_long[(e3_long['measure']=='mirroring') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='DWA')]
t_stat3, p_value3 = ttest_ind(dwa_fc_segmentation['value'], dwa_fc_masters['value'])
print(t_stat3, p_value3)
t_stat4, p_value4 = ttest_ind(dwa_fc_segmentation['value'], dwa_fc_mirroring['value'])
print(t_stat4, p_value4)

In [ ]:
bug0_fc_segmentation = e3_long[(e3_long['measure']=='segmentation') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='BUG0')]
bug0_fc_masters = e3_long[(e3_long['measure']=='masters') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='BUG0')]
bug0_fc_mirroring = e3_long[(e3_long['measure']=='mirroring') & (e3_long['metric']=='Final Correctness') & (e3_long['alg']=='BUG0')]
t_stat5, p_value5 = ttest_ind(bug0_fc_segmentation['value'], bug0_fc_masters['value'])
print(t_stat5, p_value5)
t_stat6, p_value6 = ttest_ind(bug0_fc_segmentation['value'], bug0_fc_mirroring['value'])
print(t_stat6, p_value6)